In [1]:
import matplotlib.pyplot as plt
import pyspark as ps
# from pyspark import SparkContext
# from pyspark.sql import Row
# import json
# from datetime import datetime
import numpy as np
# from scipy.stats import chisquare

plt.style.use('ggplot')
%matplotlib inline

In [2]:
spark = (ps.sql.SparkSession.builder
         .master("local[2]")
         .appName("Police Data EDA")
         .getOrCreate()
         )

rdd = spark.sparkContext.textFile('police_data.csv')
# prior_count = rdd.count()
# rdd = rdd.map(lambda x: json_not_null(x)).filter(lambda x: x is not None)
# post_count = rdd.count()

Exception: Java gateway process exited before sending its port number